# GNN Model using GCN Conv and minibatch and linear layers instead of dotproduct

In [ ]:
from torch.nn import Bilinear, Flatten

In [ ]:
class GNN_GCNConv_homogen(torch.nn.Module):
    '''
    basic implementation of GNN, inspired by content shown in https://antoniolonga.github.io/Pytorch_geometric_tutorials/posts/post12.html from Antionio Longa
    '''
    def __init__(self):
        super(GNN_homogen_with_data_bil, self).__init__()
        self.conv1 = GCNConv(data.num_features, int(data.num_features/2))
        self.conv2 = GCNConv(int(data.num_features/2), 64)
        self.bilin = Bilinear(64, 64, 1)
        self.endflat = Flatten(0,-1)
    
    def encode(self):
        x = self.conv1(data.x.relu(), data.train_pos_edge_index) # first convolution layer
        x = x.relu() # relu function for tu - disables negative values
        x = self.conv2(x, data.train_pos_edge_index) # second convolution layer
        return x
    
    def decode(self, z, pos_edge_index, neg_edge_index):
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) #fuse pos and neg edges together
        logits = self.endflat(self.bilin(z[edge_index[0]], z[edge_index[1]]))
        return logits

In [1]:
import torch
from torch_geometric.data import Data

In [2]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [3]:
attributes = torch.tensor([[1,2],[2,3],[3,4],[4,5]], dtype=torch.float)

In [4]:
y = torch.tensor([1,2,3,4], dtype=torch.long)

In [5]:
data = Data(x=x, edge_index=edge_index.contiguous(), edge_attr=attributes, y=y)
data

Data(x=[3, 1], edge_index=[2, 4], edge_attr=[4, 2], y=[4])

In [6]:
from torch_geometric.loader import NeighborLoader

In [33]:
loader = NeighborLoader(
    data,
    num_neighbors=[2],
    # Use a batch size of 128 for sampling training nodes
    batch_size=2,
)

In [27]:
zwi = next(iter(loader))

In [28]:
zwi

Data(x=[3, 1], edge_index=[2, 3], edge_attr=[3, 2], y=[3], batch_size=2)

In [34]:
count = 0
for i in loader:
    count += i.edge_index.size(1)

In [35]:
count

4

In [32]:
zwi.edge_index.size(1)

3